In [1]:
# ===============================================
# LIGHT PREPROCESSING + FACE DETECTION PIPELINE (Linked with AccoID & Timestamp)
# ===============================================
from deepface import DeepFace
import cv2, os, re
import numpy as np
from tqdm import tqdm

# === CONFIGURATION ===
INPUT_DIR = "../data/crops_person/cafe_POS_person_crops_2025_11_05"
OUTPUT_DIR = "../data/crops_face/20251105/1_cafe_pos_person_faces_yunet"
os.makedirs(OUTPUT_DIR, exist_ok=True)

DETECTOR = "yunet"  # Best for CCTV / varied angles
ENFORCE_DETECTION = True
MIN_CONFIDENCE = 0.65
TARGET_MIN = 192
TARGET_MAX = 384
EXPAND_RATIO = 0.5  # Include gentle context (hair/shoulders)

# === LIGHT IMAGE PREPROCESSING ===
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

def preprocess_soft(img):
    """Subtle enhancement for better detection without altering tone."""
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Gentle gamma correction
    gamma = 1.08
    lut = np.array([((i / 255.0) ** (1 / gamma)) * 255 for i in np.arange(256)]).astype("uint8")
    img = cv2.LUT(img, lut)

    # Mild denoising
    img = cv2.fastNlMeansDenoisingColored(img, None, 3, 3, 5, 15)
    return img

# === FILENAME PARSING FUNCTION ===
def parse_filename_for_info(filename):
    """
    Expected pattern: AccoID_1147705_20251019_083301_person_0.jpg
    Extract AccoID and timestamp.
    """
    match = re.search(r"AccoID_(\d+)_(\d{8}_\d{6})", filename)
    if match:
        acco_id = match.group(1)
        timestamp = match.group(2)
    else:
        acco_id, timestamp = "unknown", "00000000_000000"
    return acco_id, timestamp

# === PROCESSING LOOP ===
images = sorted([f for f in os.listdir(INPUT_DIR) if f.lower().endswith((".jpg", ".png"))])
print(f"🧠 Processing {len(images)} person crops...")

for i, img_name in enumerate(tqdm(images)):
    img_path = os.path.join(INPUT_DIR, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    acco_id, timestamp = parse_filename_for_info(img_name)
    img = preprocess_soft(img)
    h_img, w_img = img.shape[:2]

    try:
        detections = DeepFace.extract_faces(
            img_path=img_path,
            detector_backend=DETECTOR,
            enforce_detection=ENFORCE_DETECTION,
            align=False
        )

        if not detections:
            continue

        for j, face_info in enumerate(detections):
            conf = face_info.get("confidence", 1.0)
            if conf < MIN_CONFIDENCE:
                continue

            area = face_info.get("facial_area", {})
            if not area:
                continue

            x, y, w, h = area["x"], area["y"], area["w"], area["h"]

            # --- Adaptive crop with soft padding ---
            dx, dy = int(w * EXPAND_RATIO), int(h * EXPAND_RATIO)
            x1, y1 = max(x - dx, 0), max(y - dy, 0)
            x2, y2 = min(x + w + dx, w_img), min(y + h + dy, h_img)
            face_crop = img[y1:y2, x1:x2]

            # --- Adaptive resize ---
            h, w = face_crop.shape[:2]
            scale = min(max(TARGET_MIN / min(h, w), 1.0), TARGET_MAX / max(h, w))
            new_size = (int(w * scale), int(h * scale))
            face_crop = cv2.resize(face_crop, new_size, interpolation=cv2.INTER_CUBIC)

            # --- Save with AccoID & Timestamp in name ---
            out_name = f"AccoID_{acco_id}_{timestamp}.jpg"
            out_path = os.path.join(OUTPUT_DIR, out_name)
            cv2.imwrite(out_path, face_crop, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

    except Exception as e:
        #print(f"⚠️ Error processing {img_name}: {e}")
        continue

print(f"\n✅ Enhanced face crops saved to: {OUTPUT_DIR}")
print(f"📸 Total cropped faces: {len(os.listdir(OUTPUT_DIR))}")

🧠 Processing 29214 person crops...


100%|██████████| 29214/29214 [2:11:11<00:00,  3.71it/s]  


✅ Enhanced face crops saved to: ../data/crops_face/20251105/1_cafe_pos_person_faces_yunet
📸 Total cropped faces: 7625


In [2]:
# ===============================================
# LIGHT PREPROCESSING + FACE DETECTION PIPELINE (Linked with AccoID & Timestamp)
# ===============================================
from deepface import DeepFace
import cv2, os, re
import numpy as np
from tqdm import tqdm

# === CONFIGURATION ===
INPUT_DIR = "../data/crops_person/cafe_POS_person_crops_2025_11_07"
OUTPUT_DIR = "../data/crops_face/20251107/1_cafe_pos_person_faces_yunet"
os.makedirs(OUTPUT_DIR, exist_ok=True)

DETECTOR = "yunet"  # Best for CCTV / varied angles
ENFORCE_DETECTION = True
MIN_CONFIDENCE = 0.65
TARGET_MIN = 192
TARGET_MAX = 384
EXPAND_RATIO = 0.5  # Include gentle context (hair/shoulders)

# === LIGHT IMAGE PREPROCESSING ===
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

def preprocess_soft(img):
    """Subtle enhancement for better detection without altering tone."""
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Gentle gamma correction
    gamma = 1.08
    lut = np.array([((i / 255.0) ** (1 / gamma)) * 255 for i in np.arange(256)]).astype("uint8")
    img = cv2.LUT(img, lut)

    # Mild denoising
    img = cv2.fastNlMeansDenoisingColored(img, None, 3, 3, 5, 15)
    return img

# === FILENAME PARSING FUNCTION ===
def parse_filename_for_info(filename):
    """
    Expected pattern: AccoID_1147705_20251019_083301_person_0.jpg
    Extract AccoID and timestamp.
    """
    match = re.search(r"AccoID_(\d+)_(\d{8}_\d{6})", filename)
    if match:
        acco_id = match.group(1)
        timestamp = match.group(2)
    else:
        acco_id, timestamp = "unknown", "00000000_000000"
    return acco_id, timestamp

# === PROCESSING LOOP ===
images = sorted([f for f in os.listdir(INPUT_DIR) if f.lower().endswith((".jpg", ".png"))])
print(f"🧠 Processing {len(images)} person crops...")

for i, img_name in enumerate(tqdm(images)):
    img_path = os.path.join(INPUT_DIR, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    acco_id, timestamp = parse_filename_for_info(img_name)
    img = preprocess_soft(img)
    h_img, w_img = img.shape[:2]

    try:
        detections = DeepFace.extract_faces(
            img_path=img_path,
            detector_backend=DETECTOR,
            enforce_detection=ENFORCE_DETECTION,
            align=False
        )

        if not detections:
            continue

        for j, face_info in enumerate(detections):
            conf = face_info.get("confidence", 1.0)
            if conf < MIN_CONFIDENCE:
                continue

            area = face_info.get("facial_area", {})
            if not area:
                continue

            x, y, w, h = area["x"], area["y"], area["w"], area["h"]

            # --- Adaptive crop with soft padding ---
            dx, dy = int(w * EXPAND_RATIO), int(h * EXPAND_RATIO)
            x1, y1 = max(x - dx, 0), max(y - dy, 0)
            x2, y2 = min(x + w + dx, w_img), min(y + h + dy, h_img)
            face_crop = img[y1:y2, x1:x2]

            # --- Adaptive resize ---
            h, w = face_crop.shape[:2]
            scale = min(max(TARGET_MIN / min(h, w), 1.0), TARGET_MAX / max(h, w))
            new_size = (int(w * scale), int(h * scale))
            face_crop = cv2.resize(face_crop, new_size, interpolation=cv2.INTER_CUBIC)

            # --- Save with AccoID & Timestamp in name ---
            out_name = f"AccoID_{acco_id}_{timestamp}.jpg"
            out_path = os.path.join(OUTPUT_DIR, out_name)
            cv2.imwrite(out_path, face_crop, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

    except Exception as e:
        #print(f"⚠️ Error processing {img_name}: {e}")
        continue

print(f"\n✅ Enhanced face crops saved to: {OUTPUT_DIR}")
print(f"📸 Total cropped faces: {len(os.listdir(OUTPUT_DIR))}")

🧠 Processing 27340 person crops...


100%|██████████| 27340/27340 [2:05:01<00:00,  3.64it/s]  


✅ Enhanced face crops saved to: ../data/crops_face/20251107/1_cafe_pos_person_faces_yunet
📸 Total cropped faces: 7500
